<a href="https://colab.research.google.com/github/avner8943/bootcamp/blob/master/decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import numpy as np

def test_split(data, feature, split_val):

  right = data[data[:,feature] >= split_val]

  left = data[data[:,feature] < split_val]

  return left, right


def calc_gini(left, right, classes):

  tot_counts = float(len(left) + len(right))

  gini = 0.0

  if left.shape[0] == 0 or right.shape[0] == 0: 
    
    return 1e2


  left_0 = np.count_nonzero(left[:,-1] == classes[0])/left.shape[0]
  left_1 = np.count_nonzero(left[:,-1] == classes[1])/left.shape[0]

  right_0 = np.count_nonzero(right[:,-1] == classes[0])/right.shape[0]
  right_1 = np.count_nonzero(right[:,-1] == classes[1])/right.shape[0]

  gini = (1 - (left_0**2+left_1**2))*(left.shape[0]/tot_counts) + (1 - (right_0**2+right_1**2))*(right.shape[0]/tot_counts)

  return gini



def get_split(data):

  

  classes = np.array([0,1])

  best_idx, best_val, best_gini_score, best_groups = 1e3,1e3,1e3,0

  for feature in range(data.shape[1]-1):
    for row in data:
      left, right = test_split(data, feature, row[feature])
      gini = calc_gini(left, right, classes)

      if gini < best_gini_score:
        best_idx, best_val, best_gini_score, best_groups = feature, row[feature], gini, [left, right]

  return {'feature':best_idx, 'value':best_val, 'groups':best_groups, 'gini':best_gini_score}
 



class node():

  def __init__(self, left=0, right=0, feature=0, val=0, depth=0, data=0):

  
    self.is_leaf = 0

    self.left = left
    self.right = right
    self.feature = feature
    self.val = val
    self.depth = depth
    self.data = data
    self.decision = None

    return



class tree():

  def __init__(self, node = None):
    self.root = node


def tree_split(node_ , max_depth, min_size, depth):


  
  if depth > max_depth:

    node_.is_leaf = 1

    return


  split_node = get_split(node_.data)

  if split_node['groups'][0].shape[0] == 0 or  split_node['groups'][1].shape[0] == 0:

    node_.is_leaf = 1
    (values,counts) = np.unique(node_,return_counts=True)
    ind=np.argmax(counts)
    node_.decision = values[ind]

    return


  if split_node['groups'][0].shape[0] <= min_size:
    node_.left = node(feature=split_node['feature'], val = split_node['value'], depth = depth+1, data = split_node['groups'][0])
    node_.left.is_leaf=1
    (values,counts) = np.unique(node_.left,return_counts=True)
    ind=np.argmax(counts)
    node_.left.decision = values[ind]


  else:
    node_.left = node(feature=split_node['feature'], val = split_node['value'], depth = depth+1, data = split_node['groups'][0])
    tree_split(node_.left, max_depth, min_size, depth+1)
  
  if split_node['groups'][1].shape[0] < min_size:
    node_.right = node(feature=split_node['feature'], val = split_node['value'], depth = depth+1, data = split_node['groups'][1])
    node_.right.is_leaf=1
    (values,counts) = np.unique(node_.right,return_counts=True)
    ind=np.argmax(counts)
    node_.right.decision = values[ind]


  else:
    node_.right = node(feature=split_node['feature'], val = split_node['value'], depth = depth+1, data = split_node['groups'][1])
    tree_split(node_.left, max_depth, min_size, depth+1)


    
  
  
  

def build_tree(data, max_depth, min_size):

  root = node( depth = 0, data = data)

  tree1 = tree(root)
  tree_split(tree1.root, max_depth, min_size, 0)

  return tree1


def predict(node_, row):

  print(node_)
  if node_.is_leaf == 1:

    return node_.decision

  if row[node_.feature] < node_.val:
    print('left')
    predict(node_.left, row)

  else:
    print('right')
    predict(node_.right, row)




dataset = np.array([[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]])


tree = build_tree(dataset, 5,1)
print(dataset[0,:])

#for row in dataset[0,:]:

row = dataset[0,:]
prediction = predict(tree.root, row)
print('Expected=%d, Got=%d' % (row[-1], prediction))




[2.77124472 1.78478393 0.        ]
right
left
0


AttributeError: ignored

In [67]:
print(tree.root.right.is_leaf)

0


In [140]:

  if node == 'root':
    root_split = get_split(node.data)
    root = node(root_split['groups'][0], root_split['groups'][1], root_split['index'], root_split['value'] )

# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right

# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
	# count all samples at split point
	n_instances = float(sum([len(group) for group in groups]))
	# sum weighted Gini index for each group
	gini = 0.0
	for group in groups:
		size = float(len(group))
		# avoid divide by zero
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# weight the group score by its relative size
		gini += (1.0 - score) * (size / n_instances)
	return gini

# Select the best split point for a dataset
def get_split(dataset):
	class_values = list(set(row[-1] for row in dataset))
	b_index, b_value, b_score, b_groups = 999, 999, 999, None
	for index in range(len(dataset[0])-1):
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Create a terminal node value
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)

# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left)
		split(node['left'], max_depth, min_size, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right)
		split(node['right'], max_depth, min_size, depth+1)

# Build a decision tree
def build_tree(train, max_depth, min_size):
	root = get_split(train)
	split(root, max_depth, min_size, 1)
	return root

# Print a decision tree
def print_tree(node, depth=0):
	if isinstance(node, dict):
		print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
		print_tree(node['left'], depth+1)
		print_tree(node['right'], depth+1)
	else:
		print('%s[%s]' % ((depth*' ', node)))

dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]
tree = build_tree(dataset, 5, 1)
print_tree(tree)


[X1 < 6.642]
 [X1 < 2.771]
  [0]
  [X1 < 2.771]
   [0]
   [0]
 [X1 < 7.498]
  [X1 < 7.445]
   [1]
   [1]
  [X1 < 7.498]
   [1]
   [1]


In [25]:
a = np.array([1,2,3,1,2,1,1,1,3,2,2,1])
counts = np.bincount(a)
print(np.argmax(counts))
print(counts)

1
[0 6 4 2]
